Описание проекта

Нам надо изучить базу даных крупного сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цель исследования - найти полезную информацию для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': '', # имя пользователя
'pwd': '', # пароль
'host': '',
'port': , # порт подключения
'db': ''} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
#pd.io.sql.read_sql(query, con = engine) 

In [5]:
#Исследуйте таблицы — выведите первые строки
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = ''' SELECT* 
                FROM {}
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Задача 1.

Посчитайте, сколько книг вышло после 1 января 2000 года

In [6]:
for table in ['books']:
    query = ''' SELECT COUNT (DISTINCT book_id) 
                FROM books
                WHERE publication_date > '2000-01-01';
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request)

,count
0,819


Вывод: После 1 января 2000 года было издано 819 уникальных наименований книг.

Задача 2.

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [7]:
for table in ['books']:
    query = ''' WITH
                a AS (SELECT book_id, 
                             COUNT(review_id) OVER (PARTITION BY book_id) AS reviews_number
                      FROM reviews),
                
                b AS (SELECT book_id, 
                             AVG(rating) OVER (PARTITION BY book_id) AS avg_rating
                      FROM ratings)
                      
                SELECT DISTINCT books.book_id,
                       books.title,
                       reviews_number,
                       ROUND (avg_rating, 2) AS avg_rating
                FROM books
                FULL JOIN a ON books.book_id=a.book_id
                FULL JOIN b ON books.book_id=b.book_id
                ORDER BY avg_rating DESC;
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request)

,book_id,title,reviews_number,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4.0,5.00
1,20,A Fistful of Charms (The Hollows #4),2.0,5.00
2,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.00
3,57,Act of Treason (Mitch Rapp #9),2.0,5.00
4,62,Alas Babylon,2.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,202,Drowning Ruth,3.0,2.00
997,316,His Excellency: George Washington,2.0,2.00
998,371,Junky,2.0,2.00


Вывод: В списке достаточно много книг с отличными рейтингами и несколькими отзывами.

Задача 3.

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [8]:
for table in ['books']:
    query = ''' WITH 
                a AS (SELECT publisher_id, COUNT(book_id) OVER (PARTITION BY publisher_id) AS books
                           FROM books
                           WHERE num_pages > 50)
                
                
                
                SELECT DISTINCT publisher,
                       a.books
                FROM publishers
                RIGHT JOIN a ON publishers.publisher_id=a.publisher_id
                WHERE books = ( SELECT MAX(books) FROM a);
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request.head(10))

,publisher,books
0,Penguin Books,42


Вывод:

Penguin Books - это издательство, которое выпустило наибольшее число книг толще 50 страниц 

Задача 4.

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [9]:
for table in ['books']:
    query = ''' WITH
                a AS (SELECT DISTINCT book_id, 
                             AVG(rating) AS avg_rating
                      FROM ratings
                      WHERE book_id IN (SELECT DISTINCT book_id 
                                        FROM ratings GROUP BY book_id HAVING COUNT(rating_id) >50)
                      GROUP BY book_id)
                      
                SELECT DISTINCT author,
                       ROUND (a.avg_rating, 3) AS avg_rating
                FROM authors
                JOIN books ON authors.author_id = books.author_id
                RIGHT JOIN a ON books.book_id=a.book_id
                WHERE avg_rating = (SELECT MAX(avg_rating) FROM a)
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request)   

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.415


Вывод:


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книгучитывая книги с 50 и более оценками

Задача 5.

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [10]:
for table in ['reviews']:
    query = ''' SELECT AVG(a.reviews_number)
                FROM (SELECT DISTINCT username, 
                      COUNT(review_id) AS reviews_number
                      FROM reviews
                      GROUP BY username) AS a
                WHERE username IN (SELECT DISTINCT username 
                                   FROM ratings GROUP BY username HAVING COUNT(rating) >50);
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request) 

,avg
0,24.333333


Вывод:

Самые активные пользователей, которые оставили больше 50 оценок имеют в среднем 24 обзора.